In [1]:
import pandas as pd 

In [8]:
df = pd.read_excel(r"C:\Users\tpugn\Downloads\Coord_IFN_LaRioja.xlsx")
print(df.head())

  COMUNIDAD AUTÓNOMA  PROVINCIA  ESTADILLO CLASE SUBCLASE   PARCELA  \
0           La Rioja         26          1     A        1  260001A1   
1           La Rioja         26          3     A        1  260003A1   
2           La Rioja         26          4     A        1  260004A1   
3           La Rioja         26          5     A        1  260005A1   
4           La Rioja         26          6     A        4  260006A4   

   INVENTARIO  HUSO  XED50_H30  YED50_H30    XETRS89     YETRS89  XETRS89_H30  \
0           4    30        NaN        NaN  500888.13  4720813.21    500888.13   
1           4    30        NaN        NaN  496927.07  4719787.25    496927.07   
2           4    30        NaN        NaN  502941.13  4719823.14    502941.13   
3           4    30        NaN        NaN  503938.13  4719836.13    503938.13   
4           4    30        NaN        NaN  510893.20  4719792.10    510893.20   

   YETRS89_H30  XWGS84  YWGS84  TIPO DE COORDENADA  
0   4720813.21     NaN     NaN   

In [9]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Crear capa de puntos a partir de las coordenadas ETRS89
# Crear geometría de puntos
geometry = [Point(xy) for xy in zip(df['XETRS89_H30'], df['YETRS89_H30'])]

# Crear GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:25830')

print("Capa de puntos creada exitosamente")
print(f"Número de puntos: {len(gdf)}")
print(f"CRS: {gdf.crs}")
print(f"\nPrimeros 5 puntos:")
print(gdf.head())


Capa de puntos creada exitosamente
Número de puntos: 1311
CRS: EPSG:25830

Primeros 5 puntos:
  COMUNIDAD AUTÓNOMA  PROVINCIA  ESTADILLO CLASE SUBCLASE   PARCELA  \
0           La Rioja         26          1     A        1  260001A1   
1           La Rioja         26          3     A        1  260003A1   
2           La Rioja         26          4     A        1  260004A1   
3           La Rioja         26          5     A        1  260005A1   
4           La Rioja         26          6     A        4  260006A4   

   INVENTARIO  HUSO  XED50_H30  YED50_H30    XETRS89     YETRS89  XETRS89_H30  \
0           4    30        NaN        NaN  500888.13  4720813.21    500888.13   
1           4    30        NaN        NaN  496927.07  4719787.25    496927.07   
2           4    30        NaN        NaN  502941.13  4719823.14    502941.13   
3           4    30        NaN        NaN  503938.13  4719836.13    503938.13   
4           4    30        NaN        NaN  510893.20  4719792.10    510893

In [10]:
gdf.to_file(r"E:\SAR_UVa\shps\puntos_ifn_rioja.geojson")


## Codigo de calculo de parametros estadisticos


In [ ]:
"""
Script para calcular estadísticas de series temporales con AUC del año medio
Incluye:
- Área bajo la curva (AUC) del año medio por banda
- Media y desviación estándar por estación
- ACF en lags específicos
- Q-test (Ljung-Box)
- Índices del periodograma
"""

import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy import integrate
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import acf
from pathlib import Path

# ============================================================================
# CONFIGURACIÓN
# ============================================================================

# Ruta del archivo CSV de entrada
input_file = r"E:\SAR_UVa\CSV'S\valores_coherencia_extremadura_modificado"
output_file = 'estadisticas_series_temporales_auc.csv'

# Parámetros de análisis
fs = 1.0      # 1 observación por semana
nperseg = 52  # ventana anual
freq_muestreo = 52  # frecuencia de muestreo anual (semanas)

# Lags para ACF
acf_lags = [26, 52, 78, 104, 156]

# Lags para Q-test (Ljung-Box)
qtest_lags = [1, 2, 3, 26, 52, 104, 156]

# ============================================================================
# CARGA DE DATOS
# ============================================================================

print("=" * 80)
print("ANÁLISIS DE SERIES TEMPORALES CON AUC")
print("=" * 80)

if not Path(input_file).exists():
    print(f"❌ Error: El archivo {input_file} no existe")
    exit(1)

band_weekly = pd.read_csv(input_file)
band_weekly['date'] = pd.to_datetime(band_weekly['date'])

print(f"\n✅ Datos cargados desde: {input_file}")
print(f"   Registros: {len(band_weekly)}")
print(f"   Columnas: {band_weekly.columns.tolist()}")

# Obtener bandas disponibles
bands = band_weekly['band'].unique()
print(f"   Bandas encontradas: {bands}")

# ============================================================================
# PROCESAMIENTO PRINCIPAL
# ============================================================================

results = []
total_parcelas = len(band_weekly['ID_PARCELA'].unique())
contador = 0

for fid in band_weekly['ID_PARCELA'].unique():
    contador += 1
    if contador % 50 == 0:
        print(f"Procesando parcela {contador}/{total_parcelas}...")
    
    # Obtener la especie principal para este FID
    sp_ppal = band_weekly[band_weekly['ID_PARCELA'] == fid]['SP_PPAL'].iloc[0]
    
    # Iterar por cada banda
    for band in bands:
        
        # Filtrar datos por FID y banda
        data_fid_band = band_weekly[
            (band_weekly['ID_PARCELA'] == fid) &
            (band_weekly['band'] == band)
        ].copy()
        
        if len(data_fid_band) == 0:
            continue
        
        # Serie temporal completa
        ts = (
            data_fid_band
            .groupby('date')['seasonal_mean']
            .mean()
            .dropna()
            .sort_index()
        )
        
        if len(ts) < nperseg:
            continue
        
        # Diccionario para almacenar resultados de este FID-banda
        row_data = {
            'ID_PARCELA': fid,
            'SP_PPAL': sp_ppal,
            'band': band
        }
        
        # ========== 0. ÁREA BAJO LA CURVA DEL AÑO MEDIO POR BANDA ==========
        # Obtener el año medio (agregación semanal por semana del año)
        data_fid_band['week_of_year'] = data_fid_band['date'].dt.isocalendar().week
        
        # Agrupar por semana del año y calcular la media
        yearly_mean = (
            data_fid_band
            .groupby('week_of_year')['seasonal_mean']
            .mean()
            .sort_index()
        )
        
        if len(yearly_mean) > 0:
            # Calcular AUC usando integración trapezoidal
            x = np.arange(len(yearly_mean))  # semanas 0-51
            y = yearly_mean.values
            
            # AUC con método trapezoidal
            auc = np.trapz(y, x)
            row_data['auc_yearly_mean'] = auc
            
            # AUC normalizado (dividido por número de semanas)
            row_data['auc_yearly_mean_normalized'] = auc / len(yearly_mean)
            
            # Valor promedio del año (alternativa simple)
            row_data['mean_yearly_value'] = np.mean(y)
            
            # Mínimo y máximo del año medio
            row_data['min_yearly_value'] = np.min(y)
            row_data['max_yearly_value'] = np.max(y)
            row_data['std_yearly_value'] = np.std(y)
        else:
            row_data['auc_yearly_mean'] = np.nan
            row_data['auc_yearly_mean_normalized'] = np.nan
            row_data['mean_yearly_value'] = np.nan
            row_data['min_yearly_value'] = np.nan
            row_data['max_yearly_value'] = np.nan
            row_data['std_yearly_value'] = np.nan
        
        # ========== 1. MEDIA Y DESVIACIÓN ESTÁNDAR POR ESTACIÓN ==========
        data_fid_band['season'] = data_fid_band['date'].dt.quarter.map({
            1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'
        })
        
        for season in ['Winter', 'Spring', 'Summer', 'Fall']:
            season_data = data_fid_band[data_fid_band['season'] == season]['seasonal_mean']
            row_data[f'mean_{season}'] = season_data.mean() if len(season_data) > 0 else np.nan
            row_data[f'std_{season}'] = season_data.std() if len(season_data) > 0 else np.nan
        
        # ========== 2. ACF EN LAGS ESPECÍFICOS ==========
        try:
            # Calcular ACF
            acf_values = acf(ts.values, nlags=max(acf_lags), fft=True)
            
            for lag in acf_lags:
                if lag < len(acf_values):
                    row_data[f'acf_lag_{lag}'] = acf_values[lag]
                else:
                    row_data[f'acf_lag_{lag}'] = np.nan
        except Exception as e:
            print(f"⚠️  Error ACF para ID_PARCELA={fid}, band={band}: {e}")
            for lag in acf_lags:
                row_data[f'acf_lag_{lag}'] = np.nan
        
        # ========== 3. Q-TEST (LJUNG-BOX) EN LAGS ESPECÍFICOS ==========
        try:
            # Calcular Ljung-Box test
            lb_result = acorr_ljungbox(ts.values, lags=qtest_lags, return_df=True)
            
            for lag in qtest_lags:
                if lag in lb_result.index:
                    row_data[f'qtest_stat_{lag}'] = lb_result.loc[lag, 'lb_stat']
                    row_data[f'qtest_pvalue_{lag}'] = lb_result.loc[lag, 'lb_pvalue']
                else:
                    row_data[f'qtest_stat_{lag}'] = np.nan
                    row_data[f'qtest_pvalue_{lag}'] = np.nan
        except Exception as e:
            print(f"⚠️  Error Q-test para ID_PARCELA={fid}, band={band}: {e}")
            for lag in qtest_lags:
                row_data[f'qtest_stat_{lag}'] = np.nan
                row_data[f'qtest_pvalue_{lag}'] = np.nan
        
        # ========== 4. ÍNDICES DEL PERIODOGRAMA ==========
        try:
            # Calcular periodograma con Welch
            freqs, power = welch(
                ts.values, 
                fs=fs, 
                detrend='constant',
                scaling='density'
            )
            
            # Convertir frecuencias a periodos
            mask = freqs > 0
            periodos = 1.0 / freqs[mask]
            power = power[mask]
            
            # Ciclos de interés
            ciclos = np.array([freq_muestreo/3, freq_muestreo/2, freq_muestreo])
            
            # Encontrar posiciones de los ciclos
            pos_ciclos = [np.argmin(np.abs(periodos - ciclo)) for ciclo in ciclos]
            
            # Extraer bandas de los ciclos
            bands_power = power[pos_ciclos]
            
            # 4.1 Seasonality Mode (periodo con máxima potencia)
            max_idx = np.argmax(bands_power)
            row_data['seasonality_mode'] = ciclos[max_idx]
            
            # 4.2 Fisher Kappa (max / mean)
            max_power = np.max(bands_power)
            mean_power = np.mean(bands_power)
            row_data['fisher_kappa'] = max_power / mean_power if mean_power > 0 else np.nan
            
            # 4.3 Seasonality Stability
            power_ciclo_anual_adelante = np.sum(power[pos_ciclos[2]:])
            row_data['seasonality_stability'] = (
                max_power / power_ciclo_anual_adelante 
                if power_ciclo_anual_adelante > 0 else np.nan
            )
            
            # 4.4 Plurianual Cycles
            power_plurianual = np.sum(power[:pos_ciclos[2]])
            power_total = np.sum(power)
            row_data['plurianual_cycles'] = (
                power_plurianual / power_total 
                if power_total > 0 else np.nan
            )
            
            # 4.5 Seasonality Amplitude
            row_data['seasonality_amplitude'] = max_power
            
        except Exception as e:
            print(f"⚠️  Error Periodograma para fid={fid}, band={band}: {e}")
            row_data['seasonality_mode'] = np.nan
            row_data['fisher_kappa'] = np.nan
            row_data['seasonality_stability'] = np.nan
            row_data['plurianual_cycles'] = np.nan
            row_data['seasonality_amplitude'] = np.nan
        
        # Agregar fila a resultados
        results.append(row_data)

# ============================================================================
# GUARDAR RESULTADOS
# ============================================================================

# Crear DataFrame con resultados
df_results = pd.DataFrame(results)

# Reordenar columnas
cols = ['ID_PARCELA', 'SP_PPAL', 'band'] + [col for col in df_results.columns if col not in ['ID_PARCELA', 'SP_PPAL', 'band']]
df_results = df_results[cols]

# Guardar CSV
df_results.to_csv(output_file, index=False)

print("\n" + "=" * 80)
print("✅ ANÁLISIS COMPLETADO")
print("=" * 80)
print(f"Archivo guardado en: {output_file}")
print(f"Total de registros: {len(df_results)}")
print(f"\nColumnas generadas ({len(df_results.columns)}):")
for i, col in enumerate(df_results.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\n📊 Resumen de AUC por banda:")
auc_summary = df_results.groupby('band')[['auc_yearly_mean', 'auc_yearly_mean_normalized', 'mean_yearly_value']].describe()
print(auc_summary)

print(f"\nPrimeras 5 filas:")
print(df_results.head())
